## 获取待提取特征的文件

提供两种批量处理的模式：
1. 目录模式，提取指定目录下的所有jpg文件的特征。
2. 文件模式，待提取的数据存储在文件中，每行一个样本。

当然也可以在最后自己指定手动提取指定若干文件。

In [1]:
import os
import pandas as pd
from glob import glob
from onekey_algo import get_param_in_cwd
import os

os.environ['CUDA_VISIBLE_DEVICES']='0'

# 目录模式
#data_root = os.path.join(get_param_in_cwd('data_root'), 'patches')
data_root = r'F:\TCGA-NORMAL\patches'
#data_root = r'E:\SYX\patchs'

#samples = glob(os.path.join(data_root, '*', '*.jpg'))
samples = glob(os.path.join(data_root, '*', '*.jpg'))
print(len(samples))


1928972


## 确定提取特征

通过关键词获取要提取那一层的特征。

### 支持的模型名称

模型名称替换代码中的 `model_name`变量的值。

| **模型系列** | **模型名称**                                                 |
| ------------ | ------------------------------------------------------------ |
| AlexNet      | alexnet                                                      |
| VGG          | vgg11, vgg11_bn, vgg13, vgg13_bn, vgg16, vgg16_bn, vgg19_bn, vgg19 |
| ResNet       | resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2 |
| DenseNet     | densenet121, densenet169, densenet201, densenet161           |
| Inception    | googlenet, inception_v3                                      |
| SqueezeNet   | squeezenet1_0, squeezenet1_1                                 |
| ShuffleNetV2 | shufflenet_v2_x2_0, shufflenet_v2_x0_5, shufflenet_v2_x1_0, shufflenet_v2_x1_5 |
| MobileNet    | mobilenet_v2, mobilenet_v3_large, mobilenet_v3_small         |
| MNASNet      | mnasnet0_5, mnasnet0_75, mnasnet1_0, mnasnet1_3              |

In [2]:
from onekey_algo.custom.components.comp2 import extract, print_feature_hook, reg_hook_on_module, \
    init_from_model, init_from_onekey
import os

os.environ['CUDA_VISIBLE_DEVICES']='0'

#model, transformer, device = init_from_onekey(r'F:\20230915-BianYun\models\SimpleViT\viz')

model, transformer, device = init_from_onekey(r'D:\bianyun_onekey\models\SimpleViT\viz')
print('device',device)
for n, m in model.named_modules():
    print('Feature name:', n, "|| Module:", m)

[2024-06-12 15:19:28 - simple_vit.py:  93]	INFO	正在使用 SimpleViT，具体参数为：images_size=256, patch_size=32, num_classes=2, dim=1024, depth=6, heads=16, mlp_dim=2048


True 22222222222
cuda
device cuda
Feature name:  || Module: SimpleViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=32, p2=32)
    (1): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=3072, out_features=1024, bias=True)
    (3): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (transformer): Transformer(
    (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0): ModuleList(
        (0): Attention(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attend): Softmax(dim=-1)
          (to_qkv): Linear(in_features=1024, out_features=3072, bias=False)
          (to_out): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (1): FeedForward(
          (net): Sequential(
            (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=1024, out_features

## 提取特征

`Feature name:` 之后的名称为要提取的特征名，例如`layer3.0.conv2`, 一般深度学习特征提取最后一层，例如`avgpool`

In [3]:
from functools import partial
print(device)
feature_name = 'to_latent'
with open(os.path.join(r'D:\bianyun_onekey\models\SimpleViT\viz', f'dlfeature.csv'), 'w') as outfile:
    hook = partial(print_feature_hook, fp=outfile)
    find_num = reg_hook_on_module(feature_name, model, hook)
    results = extract(samples, model, transformer, device, fp=outfile)

cuda
